In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sp-nasdaq-07-23/during_pre_post.csv


In [2]:
# !pip install pandas matplotlib seaborn scikit-learn transformers datasets

In [3]:
!pip install plotnine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.2
    Uninstalling matplotlib-3.7.2:
      Successfully uninstalled matplotlib-3.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
ydata-profiling 4.17.0 requires matplotlib<=3.10,>=3.5, but you have matplotlib 3.10.7 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [4]:
# !pip3 install torch torchvision

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertTokenizer, BertModel
from datasets import Dataset
from plotnine import *
import numpy as np
from tqdm import tqdm
import torch.nn as nn

2025-10-14 02:30:54.730530: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760409054.908365      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760409054.958700      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df = pd.read_csv("/kaggle/input/sp-nasdaq-07-23/during_pre_post.csv")

In [7]:
df.head()

,tickerdate,tickertime,index_symbol,market_phase,open,high,low,close
0,2007-10-26,11:59:00,SPX,GFC_Crash,1523.00,1523.00,1521.97,1521.97
1,2007-11-02,12:47:00,SPX,GFC_Crash,1508.00,1508.50,1508.00,1508.46
2,2007-11-21,15:37:00,SPX,GFC_Crash,1425.00,1425.00,1423.97,1424.25
3,2007-12-11,10:37:00,SPX,GFC_Crash,1515.16,1515.16,1515.00,1515.00
4,2008-02-20,15:49:00,SPX,GFC_Crash,1357.00,1357.00,1356.31,1356.61


In [8]:
# Drop column "market_phase"
df = df.drop(columns=["market_phase"])

In [9]:
# separate the dataframe based on "index_symbol"
df_list = [group for _, group in df.groupby("index_symbol")]

In [10]:
spx_df = df[df['index_symbol'] == 'SPX']
ndx_df = df[df['index_symbol'] == 'NDX']

In [11]:
spx_df.head()

,tickerdate,tickertime,index_symbol,open,high,low,close
0,2007-10-26,11:59:00,SPX,1523.00,1523.00,1521.97,1521.97
1,2007-11-02,12:47:00,SPX,1508.00,1508.50,1508.00,1508.46
2,2007-11-21,15:37:00,SPX,1425.00,1425.00,1423.97,1424.25
3,2007-12-11,10:37:00,SPX,1515.16,1515.16,1515.00,1515.00
4,2008-02-20,15:49:00,SPX,1357.00,1357.00,1356.31,1356.61


In [12]:
spx_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1537664 entries, 0 to 1537663
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   tickerdate    1537664 non-null  object 
 1   tickertime    1537664 non-null  object 
 2   index_symbol  1537664 non-null  object 
 3   open          1537664 non-null  float64
 4   high          1537664 non-null  float64
 5   low           1537664 non-null  float64
 6   close         1537664 non-null  float64
dtypes: float64(4), object(3)
memory usage: 93.9+ MB


In [13]:
spx = spx_df.copy()

In [14]:
spx["datetime"] = pd.to_datetime(spx["tickerdate"] + " " + spx["tickertime"])
spx = spx.sort_values("datetime")

In [15]:
spx = spx.set_index("datetime")[["open","high","low","close"]]

In [16]:
SESSION_START = "09:30"
SESSION_END   = "16:00"

is_weekday = spx.index.weekday < 5
hhmm = spx.index.strftime("%H:%M")
in_session = (hhmm >= SESSION_START) & (hhmm <= SESSION_END)

spx = spx.loc[is_weekday & in_session].copy()

In [17]:
# filter only the last 5 years
end_dt = spx.index.max()
cutoff = end_dt - pd.DateOffset(years=5)
spx = spx.loc[spx.index >= cutoff].copy()

print("Date range:", spx.index.min(), "→", spx.index.max())
print("Rows after 5y filter:", len(spx))

Date range: 2018-03-26 09:30:00 → 2023-03-24 16:00:00
Rows after 5y filter: 490637


In [18]:
spx["log_return"] = np.log(spx["close"] / spx["close"].shift(1))

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log


In [19]:
# time features (cyclic encodings, as used in Informer/FEDformer-style setups)
spx["month"]   = spx.index.month
spx["weekday"] = spx.index.weekday
spx["hour"]    = spx.index.hour
spx["minute"]  = spx.index.minute

In [20]:
spx["month_sin"]   = np.sin(2*np.pi*spx["month"]/12)
spx["month_cos"]   = np.cos(2*np.pi*spx["month"]/12)
spx["weekday_sin"] = np.sin(2*np.pi*spx["weekday"]/7)
spx["weekday_cos"] = np.cos(2*np.pi*spx["weekday"]/7)
spx["hour_sin"]    = np.sin(2*np.pi*spx["hour"]/24)
spx["hour_cos"]    = np.cos(2*np.pi*spx["hour"]/24)
spx["minute_sin"]  = np.sin(2*np.pi*spx["minute"]/60)
spx["minute_cos"]  = np.cos(2*np.pi*spx["minute"]/60)

In [21]:
#drop the first NaN from log_return
spx = spx.dropna(subset=["log_return"]).copy()

In [22]:
TIME_FEATS = [
    "month_sin","month_cos",
    "weekday_sin","weekday_cos",
    "hour_sin","hour_cos",
    "minute_sin","minute_cos"
]

In [23]:
# model will see past log_return + these time features
MODEL_FEATS = ["log_return"] + TIME_FEATS

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
# sort te dataframe
spx = spx.sort_index()

In [26]:
# list unique trading days 
trading_days = np.array(sorted({ts.date() for ts in spx.index}))

In [27]:
# compute day-based boundaries
D = len(trading_days)
d_train_end = int(D * 0.70)
d_val_end   = int(D * 0.80)

In [28]:
# data split based on the day boundaries
train_days = trading_days[:d_train_end]
val_days   = trading_days[d_train_end:d_val_end]
test_days  = trading_days[d_val_end:]

In [29]:
# slice by full days (each split starts at that day’s first bar)
spx_train = spx[np.isin(spx.index.date, train_days)].copy()
spx_val   = spx[np.isin(spx.index.date, val_days)].copy()
spx_test  = spx[np.isin(spx.index.date, test_days)].copy()

In [30]:
# scale ONLY log_return using TRAIN statistics
scaler = StandardScaler().fit(spx_train[["log_return"]])
spx_train["log_return"] = scaler.transform(spx_train[["log_return"]])
spx_val["log_return"]   = scaler.transform(spx_val[["log_return"]])
spx_test["log_return"]  = scaler.transform(spx_test[["log_return"]])

In [31]:
X_train = spx_train[MODEL_FEATS].to_numpy(dtype="float32")
X_val   = spx_val  [MODEL_FEATS].to_numpy(dtype="float32")
X_test  = spx_test [MODEL_FEATS].to_numpy(dtype="float32")


In [32]:
# sanity check: first timestamps should be around 09:30 (or first available bar)
print("Train start:", spx_train.index.min())
print("Val   start:", spx_val.index.min())
print("Test  start:", spx_test.index.min())
print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

Train start: 2018-03-26 09:31:00
Val   start: 2021-09-23 09:30:00
Test  start: 2022-03-24 09:30:00
Shapes: (343198, 9) (49107, 9) (98331, 9)


In [33]:
# Choosing the context window
Tin  = 330
Tout = 27

def make_windows(arr: np.ndarray, Tin: int, Tout: int):
    X_enc, X_dec, Y = [], [], []
    T, D = arr.shape
    last_start = T - (Tin + Tout)
    if last_start <= 0:
        raise ValueError("Not enough rows for the chosen Tin/Tout.")

    for i in range(last_start):
        enc = arr[i:i+Tin]                 
        tgt = arr[i+Tin:i+Tin+Tout]         
        start = np.zeros((1, D), dtype=np.float32)
        dec_in = np.vstack([start, tgt[:-1]])
        X_enc.append(enc)
        X_dec.append(dec_in)
        Y.append(tgt)

    return (np.array(X_enc, dtype=np.float32),
            np.array(X_dec, dtype=np.float32),
            np.array(Y,     dtype=np.float32))

In [34]:
# build arrays from your 70:10:20 day-aligned splits
X_train_np = spx_train[MODEL_FEATS].to_numpy(dtype=np.float32)
X_val_np   = spx_val  [MODEL_FEATS].to_numpy(dtype=np.float32)
X_test_np  = spx_test [MODEL_FEATS].to_numpy(dtype=np.float32)

tr_enc, tr_dec, tr_tgt = make_windows(X_train_np, Tin, Tout)
va_enc, va_dec, va_tgt = make_windows(X_val_np,   Tin, Tout)
te_enc, te_dec, te_tgt = make_windows(X_test_np,  Tin, Tout)

print("Train windows:", tr_enc.shape, tr_dec.shape, tr_tgt.shape)
print("Val   windows:", va_enc.shape, va_dec.shape, va_tgt.shape)
print("Test  windows:", te_enc.shape, te_dec.shape, te_tgt.shape)

Train windows: (342841, 330, 9) (342841, 27, 9) (342841, 27, 9)
Val   windows: (48750, 330, 9) (48750, 27, 9) (48750, 27, 9)
Test  windows: (97974, 330, 9) (97974, 27, 9) (97974, 27, 9)


In [35]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [36]:
# convert numpy windows to PyTorch tensors
tr_enc_t  = torch.from_numpy(tr_enc)   
tr_dec_t  = torch.from_numpy(tr_dec)   
tr_tgt_t  = torch.from_numpy(tr_tgt)  

va_enc_t  = torch.from_numpy(va_enc)
va_dec_t  = torch.from_numpy(va_dec)
va_tgt_t  = torch.from_numpy(va_tgt)


In [37]:
# datasets
train_ds = TensorDataset(tr_enc_t, tr_dec_t, tr_tgt_t)
val_ds   = TensorDataset(va_enc_t, va_dec_t, va_tgt_t)


In [38]:
# loaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, drop_last=False)

In [39]:
import math
import torch.nn as nn

D = tr_enc.shape[2]   

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=4096, dropout=0.1):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe)
    def forward(self, x):  # x: [S, B, E]
        return self.drop(x + self.pe[:x.size(0)].unsqueeze(1))

class TinyTransformer(nn.Module):
    def __init__(self, d_in, d_model=64, nhead=4, enc_layers=2, dec_layers=2, ff=128, dropout=0.1):
        super().__init__()
        self.inp = nn.Linear(d_in, d_model)
        self.pos = PositionalEncoding(d_model, dropout=dropout)
        self.tf  = nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=enc_layers, num_decoder_layers=dec_layers,
            dim_feedforward=ff, dropout=dropout, batch_first=False
        )
        self.out = nn.Linear(d_model, d_in)

    @staticmethod
    def tgt_mask(sz, device):
        m = torch.triu(torch.ones(sz, sz, device=device), diagonal=1).bool()
        return torch.zeros(sz, sz, device=device).masked_fill(m, float('-inf'))

    def forward(self, src, tgt):  # src: [Tin,B,D], tgt: [Tout,B,D]
        src = self.pos(self.inp(src))
        tgt = self.pos(self.inp(tgt))
        mask = self.tgt_mask(tgt.size(0), tgt.device)
        mem  = self.tf.encoder(src)
        dec  = self.tf.decoder(tgt, mem, tgt_mask=mask)
        return self.out(dec)  # [Tout,B,D]

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = TinyTransformer(d_in=D).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

EPOCHS = 10  

for epoch in range(1, EPOCHS+1):
    # train
    model.train()
    train_loss, n_train = 0.0, 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d} [Train]", leave=False)
    for enc, dec_in, tgt in pbar:
        enc = enc.transpose(0,1).to(device)
        dec = dec_in.transpose(0,1).to(device)
        tgt = tgt.transpose(0,1).to(device)

        opt.zero_grad()
        out = model(enc, dec)
        loss = loss_fn(out, tgt)
        loss.backward()
        opt.step()

        train_loss += loss.item() * enc.size(1)
        n_train    += enc.size(1)
        pbar.set_postfix({"batch_loss": f"{loss.item():.6f}"})

    train_loss /= n_train

    # validate
    model.eval()
    val_loss, n_val = 0.0, 0
    pbar = tqdm(val_loader, desc=f"Epoch {epoch:02d} [Val]", leave=False)
    with torch.no_grad():
        for enc, dec_in, tgt in pbar:
            enc = enc.transpose(0,1).to(device)
            dec = dec_in.transpose(0,1).to(device)
            tgt = tgt.transpose(0,1).to(device)

            out = model(enc, dec)
            vloss = loss_fn(out, tgt)

            val_loss += vloss.item() * enc.size(1)
            n_val    += enc.size(1)
            pbar.set_postfix({"batch_loss": f"{vloss.item():.6f}"})

    val_loss /= n_val
    print(f"Epoch {epoch:02d} | train MSE={train_loss:.6f} | val MSE={val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)


Epoch 01 | train MSE=0.117676 | val MSE=0.084829


Epoch 02 | train MSE=0.112937 | val MSE=0.083993


Epoch 03 | train MSE=0.112296 | val MSE=0.084664


Epoch 04 | train MSE=0.112131 | val MSE=0.084686


...

Epoch 06 | train MSE=0.112559 | val MSE=0.085247


Epoch 07 | train MSE=0.112379 | val MSE=0.084928


Epoch 08 | train MSE=0.113038 | val MSE=0.085084


Epoch 09 | train MSE=0.112025 | val MSE=0.084019


Epoch 10 | train MSE=0.112864 | val MSE=0.085510


In [49]:
torch.save(model.state_dict(), "vanilla_transformer_330_27.pth")

In [50]:
import joblib
joblib.dump(scaler, "scaler_log_return.joblib")

['scaler_log_return.joblib']

In [52]:
scaler = joblib.load("scaler_log_return.joblib")

In [53]:
@torch.no_grad()
def forecast_next(model, arr_tail, steps, device):
    """
    arr_tail: np.array of shape [Tin, D] (normalized features)
    steps:    Tout
    returns:  np.array of shape [Tout, D] (normalized predictions)
    """
    model.eval()
    x = torch.from_numpy(arr_tail.astype(np.float32)).unsqueeze(1).to(device)  
    start = torch.zeros(1, 1, arr_tail.shape[1], device=device)                
    # encode once
    mem = model.tf.encoder(model.pos(model.inp(x)))
    dec_in = start
    preds = []
    for _ in range(steps):
        de  = model.pos(model.inp(dec_in))
        m   = model.tgt_mask(de.size(0), de.device)
        out = model.tf.decoder(de, mem, tgt_mask=m)
        y   = model.out(out[-1:])  # [1,1,D]
        preds.append(y)
        dec_in = torch.cat([dec_in, y], dim=0)
    return torch.cat(preds, dim=0).squeeze(1).cpu().numpy() 

In [54]:
def directional_accuracy(y_true, y_pred, col=0):
    return float((np.sign(y_true[:, col]) == np.sign(y_pred[:, col])).mean())

mse_list, mae_list, da_list = [], [], []

for i in range(len(te_enc)):
    yhat = forecast_next(model, te_enc[i], steps=Tout, device=device) 
    ytrue = te_tgt[i]                                                 
    mse = np.mean((ytrue - yhat)**2)
    mae = np.mean(np.abs(ytrue - yhat))
    da  = directional_accuracy(ytrue, yhat, col=0) 
    mse_list.append(mse); mae_list.append(mae); da_list.append(da)

print(f"Test MSE (avg): {np.mean(mse_list):.6f}")
print(f"Test MAE (avg): {np.mean(mae_list):.6f}")
print(f"Directional Accuracy on log_return (avg): {np.mean(da_list):.3f}")

KeyboardInterrupt: 

In [58]:
# import torch
# import numpy as np

# @torch.no_grad()
# def evaluate_model(model, te_enc, te_tgt, Tout, device):
#     model.eval()
#     mse_list, mae_list, da_list = [], [], []

#     for i in range(len(te_enc)):
#         # Convert input to GPU tensor
#         enc_tail = torch.from_numpy(te_enc[i].astype(np.float32)).to(device)

#         # Get forecast directly in torch (keep GPU)
#         yhat_torch = forecast_next(model, te_enc[i], steps=Tout, device=device)
#         yhat = torch.from_numpy(yhat_torch).to(device)
#         ytrue = torch.from_numpy(te_tgt[i].astype(np.float32)).to(device)

#         # Compute metrics (all on GPU)
#         mse = torch.mean((ytrue - yhat) ** 2).item()
#         mae = torch.mean(torch.abs(ytrue - yhat)).item()
#         da  = torch.mean((torch.sign(ytrue[:, 0]) == torch.sign(yhat[:, 0])).float()).item()

#         mse_list.append(mse)
#         mae_list.append(mae)
#         da_list.append(da)

#         del yhat, ytrue, enc_tail
#         torch.cuda.empty_cache()  # optional cleanup for memory safety

#     print(f"Test MSE (avg): {np.mean(mse_list):.6f}")
#     print(f"Test MAE (avg): {np.mean(mae_list):.6f}")
#     print(f"Directional Accuracy (avg): {np.mean(da_list):.3f}")
import torch
import numpy as np
from tqdm.notebook import tqdm   # notebook-friendly progress bar

@torch.no_grad()
def evaluate_model(model, te_enc, te_tgt, Tout, device):
    model.eval()
    mse_list, mae_list, da_list = [], [], []

    # tqdm shows progress + estimated time
    for i in tqdm(range(len(te_enc)), desc="Evaluating test samples"):
        # Convert input to GPU tensor
        enc_tail = torch.from_numpy(te_enc[i].astype(np.float32)).to(device)

        # Get forecast directly in torch (keep GPU)
        yhat_torch = forecast_next(model, te_enc[i], steps=Tout, device=device)
        yhat = torch.from_numpy(yhat_torch).to(device)
        ytrue = torch.from_numpy(te_tgt[i].astype(np.float32)).to(device)

        # Compute metrics (all on GPU)
        mse = torch.mean((ytrue - yhat) ** 2).item()
        mae = torch.mean(torch.abs(ytrue - yhat)).item()
        da  = torch.mean((torch.sign(ytrue[:, 0]) == torch.sign(yhat[:, 0])).float()).item()

        mse_list.append(mse)
        mae_list.append(mae)
        da_list.append(da)

        # free memory
        del yhat, ytrue, enc_tail
        torch.cuda.empty_cache()

    # after loop finishes
    print("\n✅ Evaluation completed:")
    print(f"Test MSE (avg): {np.mean(mse_list):.6f}")
    print(f"Test MAE (avg): {np.mean(mae_list):.6f}")
    print(f"Directional Accuracy (avg): {np.mean(da_list):.3f}")

In [59]:
evaluate_model(model, te_enc, te_tgt, Tout, device)

Evaluating test samples:   0%|          | 0/97974 [00:00<?, ?it/s]


✅ Evaluation completed:
Test MSE (avg): 0.139838
Test MAE (avg): 0.108471
Directional Accuracy (avg): 0.499


In [41]:
# !nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-78c96ead-07f6-79f6-2336-40485d2b2e96)


In [42]:
# import torch
# print("GPUs available:", torch.cuda.device_count())
# print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

GPUs available: 1
GPU name: Tesla P100-PCIE-16GB


In [43]:
# import torch
# from torch import nn
# from torch.cuda.amp import autocast, GradScaler
# from tqdm import tqdm

# # ---------------------------
# # 1. GPU setup
# # ---------------------------
# if torch.cuda.is_available():
#     print(f"GPUs available: {torch.cuda.device_count()}")
#     for i in range(torch.cuda.device_count()):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
# else:
#     raise SystemError("No CUDA devices found.")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # ---------------------------
# # 2. Model setup
# # ---------------------------
# model = TinyTransformer(d_in=D)
# if torch.cuda.device_count() > 1:
#     # Parallelize across all GPUs
#     model = nn.DataParallel(model)
# model = model.to(device)

# opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
# loss_fn = nn.MSELoss()
# scaler = GradScaler()

# torch.backends.cudnn.benchmark = True
# torch.set_float32_matmul_precision("high")

# # ---------------------------
# # 3. Training loop
# # ---------------------------
# EPOCHS = 10

# for epoch in range(1, EPOCHS + 1):
#     model.train()
#     train_loss, n_train = 0.0, 0

#     pbar = tqdm(train_loader, desc=f"Epoch {epoch:02d} [Train]")
#     for enc, dec_in, tgt in pbar:
#         enc = enc.transpose(0, 1).to(device, non_blocking=True)
#         dec = dec_in.transpose(0, 1).to(device, non_blocking=True)
#         tgt = tgt.transpose(0, 1).to(device, non_blocking=True)

#         opt.zero_grad(set_to_none=True)

#         with autocast():  # mixed precision
#             out = model(enc, dec)
#             loss = loss_fn(out, tgt)

#         scaler.scale(loss).backward()
#         scaler.step(opt)
#         scaler.update()

#         train_loss += loss.item() * enc.size(1)
#         n_train += enc.size(1)
#         pbar.set_postfix({"batch_loss": f"{loss.item():.6f}"})

#         del loss, out  # release GPU memory

#     train_loss /= n_train

#     # ------------------ Validation ------------------
#     model.eval()
#     val_loss, n_val = 0.0, 0
#     pbar = tqdm(val_loader, desc=f"Epoch {epoch:02d} [Val]")
#     with torch.inference_mode(), autocast():
#         for enc, dec_in, tgt in pbar:
#             enc = enc.transpose(0, 1).to(device, non_blocking=True)
#             dec = dec_in.transpose(0, 1).to(device, non_blocking=True)
#             tgt = tgt.transpose(0, 1).to(device, non_blocking=True)

#             out = model(enc, dec)
#             vloss = loss_fn(out, tgt)

#             val_loss += vloss.item() * enc.size(1)
#             n_val += enc.size(1)
#             pbar.set_postfix({"batch_loss": f"{vloss.item():.6f}"})

#             del vloss, out

#     val_loss /= n_val
#     print(f"Epoch {epoch:02d} | train MSE={train_loss:.6f} | val MSE={val_loss:.6f}")

In [44]:
# # If you're on GCP VM/Vertex/Colab TPU runtime these may already be present.
# !pip -q install torch torch_xla==2.3 -f https://storage.googleapis.com/libtpu-releases/index.html

# import os
# os.environ["PJRT_DEVICE"] = "TPU"     # use PJRT runtime on TPU v5e
# os.environ["XLA_USE_BF16"] = "1"      # bfloat16 for speed + memory
# os.environ["XLA_IR_DEBUG"] = "0"
# os.environ["XLA_EXPERIMENTAL"] = "nonzero:masked_select_lowering"  # minor perf tweak

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━

In [45]:
# import os
# import math
# import torch
# from torch import nn
# from torch.utils.data import DataLoader, DistributedSampler

# # XLA imports
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

# # --------------------------
# # Configs (tweak as needed)
# # --------------------------
# EPOCHS = 10
# PER_CORE_BATCH = 64    # try 64–256; increase until ~80–90% HBM used
# NUM_WORKERS = 4        # tune (2–8) based on your VM CPU
# PRINT_EVERY = 1        # epochs

# # --------------------------
# # Your model / loss / opt
# # --------------------------
# def build_model_and_optim():
#     model = TinyTransformer(d_in=D)
#     loss_fn = nn.MSELoss()
#     opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
#     return model, loss_fn, opt

# # --------------------------
# # Dataloaders per core
# # --------------------------
# def build_loaders():
#     # IMPORTANT: per-core sampler so each TPU core sees unique data
#     train_sampler = DistributedSampler(
#         train_dataset,
#         num_replicas=xm.xrt_world_size(),
#         rank=xm.get_ordinal(),
#         shuffle=True,
#         drop_last=False,
#     )
#     val_sampler = DistributedSampler(
#         val_dataset,
#         num_replicas=xm.xrt_world_size(),
#         rank=xm.get_ordinal(),
#         shuffle=False,
#         drop_last=False,
#     )

#     train_loader = DataLoader(
#         train_dataset,
#         batch_size=PER_CORE_BATCH,
#         sampler=train_sampler,
#         num_workers=NUM_WORKERS,
#         pin_memory=False,     # pin_memory is a CUDA thing; off for TPU
#         persistent_workers=True,
#         drop_last=False,
#     )
#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=PER_CORE_BATCH,
#         sampler=val_sampler,
#         num_workers=NUM_WORKERS,
#         pin_memory=False,
#         persistent_workers=True,
#         drop_last=False,
#     )
#     return train_loader, val_loader, train_sampler, val_sampler

# # --------------------------
# # Train / Val loops on TPU
# # --------------------------
# def train_loop(rank):
#     # Each process is bound to one TPU core
#     device = xm.xla_device()

#     model, loss_fn, opt = build_model_and_optim()
#     model = model.to(device)

#     train_loader, val_loader, train_sampler, val_sampler = build_loaders()

#     # XLA wraps DataLoader so each core gets a device-bound iterator
#     train_xla = pl.MpDeviceLoader(train_loader, device)
#     val_xla   = pl.MpDeviceLoader(val_loader,   device)

#     for epoch in range(1, EPOCHS + 1):
#         # Ensure different shuffles per epoch across cores
#         train_sampler.set_epoch(epoch)
#         val_sampler.set_epoch(epoch)

#         # ---------------- Train ----------------
#         model.train()
#         train_loss_sum = 0.0
#         train_count    = 0

#         for enc, dec_in, tgt in train_xla:
#             # Keep logic identical, just on TPU device
#             enc = enc.transpose(0, 1)  # [T,B,...]
#             dec = dec_in.transpose(0, 1)
#             tgt = tgt.transpose(0, 1)

#             opt.zero_grad(set_to_none=True)

#             # BF16 is enabled globally via XLA_USE_BF16, so no autocast needed
#             out  = model(enc, dec)
#             loss = loss_fn(out, tgt)

#             loss.backward()
#             # Use XLA optimizer step (executes graph & syncs grads)
#             xm.optimizer_step(opt, barrier=True)

#             # Local sums (avoid .item() in loop to reduce host syncs)
#             bsz = enc.size(1)
#             train_loss_sum += float(loss.detach()) * bsz
#             train_count    += int(bsz)

#             # Early delete to keep memory tidy
#             del loss, out, enc, dec, tgt

#         # Cross-core reduction to get global mean
#         train_loss_sum = xm.mesh_reduce('train_loss_sum', train_loss_sum, sum)
#         train_count    = xm.mesh_reduce('train_count',    train_count,    sum)
#         train_mse = train_loss_sum / max(train_count, 1)

#         # ---------------- Validate ----------------
#         model.eval()
#         val_loss_sum = 0.0
#         val_count    = 0
#         with torch.no_grad():
#             for enc, dec_in, tgt in val_xla:
#                 enc = enc.transpose(0, 1)
#                 dec = dec_in.transpose(0, 1)
#                 tgt = tgt.transpose(0, 1)

#                 out   = model(enc, dec)
#                 vloss = loss_fn(out, tgt)

#                 bsz = enc.size(1)
#                 val_loss_sum += float(vloss.detach()) * bsz
#                 val_count    += int(bsz)

#                 del vloss, out, enc, dec, tgt

#         val_loss_sum = xm.mesh_reduce('val_loss_sum', val_loss_sum, sum)
#         val_count    = xm.mesh_reduce('val_count',    val_count,    sum)
#         val_mse = val_loss_sum / max(val_count, 1)

#         # Print only once (core 0)
#         if xm.is_master_ordinal() and (epoch % PRINT_EVERY == 1 or epoch == EPOCHS):
#             print(f"Epoch {epoch:02d} | train MSE={train_mse:.6f} | val MSE={val_mse:.6f}")

#     # Optional barrier to ensure all cores finish before exiting
#     xm.rendezvous('training_done')

# # ------------- Launch 8 TPU processes -------------
# # v5e-8 has 8 chips/cores → spawn 8 workers
# xmp.spawn(train_loop, args=(), nprocs=None)

ModuleNotFoundError: No module named 'torch_xla'